# Some imports

In [ ]:
import os
import subprocess

# Paths and parameters

In [ ]:
experiment_dir = '/scratch/owb/Downloads/amdtk/recipes/wsj_mllt_fmllr_lda'
segmentation_ctm = 'ploop_mfcc_c1_T100_sil0_s3_g2_a3_b3/bigram_ws/6_0.5_10/KnownN_1_UnkN_2/TimedSentences_Iter_150'
wav_scp = 'data/training_unique.scp'
split_file = 'data/wsj.split'
sph2pipe_bin = '/scratch/owb/Downloads/amdtk/tools/sph2pipe_v2.5/sph2pipe'

# Convert segmentation result from CTM to cluster file

In [ ]:
def convert_segmentation(infile, outfile, segments_file, experiment_dir):
    convert_cmd = 'convert_ctm_to_eval2_cluster'
    path_sh = os.path.join(experiment_dir, 'path.sh')
    cmd = 'source {} && {}'.format(path_sh, convert_cmd)
    if segments_file is not None:
        cmd += ' -s {}'.format(segments_file)
    cmd += ' {} {}'.format(infile, outfile)
    subprocess.call([cmd], shell=True)

In [ ]:
infile = os.path.join(experiment_dir, segmentation_ctm)
outfile = '{}.classes'.format(infile)
segments_file = os.path.join(experiment_dir, split_file)
convert_segmentation(infile, outfile, segments_file, experiment_dir)

# Read wav file list mapping from key to filename

In [ ]:
def read_wav_scp(wav_file):
    wav_list = dict()
    with open(wav_file) as fid:
        for entry in fid.readlines():
            file_descriptor, key = entry.strip().split(':')
            file = file_descriptor.split('[')[0]
            wav_list[key] = file
    return wav_list

In [ ]:
wav_list = read_wav_scp(os.path.join(experiment_dir, wav_scp))

# Split wav files into short segments and place into corresponding directory

In [ ]:
out_dir = '{}_wavs'.format(outfile)
with open(outfile) as fid:
    for line in fid.readlines():
        line_split = line.split()
        if len(line_split) == 2:
            dir_name = os.path.join(out_dir, '_'.join(line_split[0:2]))
            os.makedirs(dir_name, exist_ok=True)
        if len(line_split) == 3:
            wav_file = wav_list[line_split[0]]
            start = line_split[1]
            end = line_split[2]
            segment_file = os.path.join(dir_name, '_'.join(line_split) + '.wav')
            args = [sph2pipe_bin, '-f', 'wav', '-t', start + ':' + end, wav_file, segment_file]
            subprocess.call(args)